In [ ]:
import pandas as pd
import string
import re
import pymongo
#from pymongo import MongoClient 
from nltk.tokenize import TweetTokenizer 

client = pymongo.MongoClient('mongodb://TwitterIPO?authSource=TwitterIPO')

In [ ]:
def clean_tweet(t):

    t = t.replace('RT ','')
    t = t.replace('$','')
    t = t.replace('#','')
    
    for word in t:
        if word.startswith(("@")) == True:
            t = t.replace(word,'')
            
    t = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''','',t)

    t = t.replace(',','')
    t = t.replace('.','')
    t = t.replace(';','')
    t = t.replace("'",'')
    t = t.replace('"','')
    t = t.replace('...','')
    
    return(t.lower())

In [ ]:
listofAlphabet = list(string.ascii_lowercase)

alphabetDictionary = {}

alphabetIntegerValue = 1

for alphabet in listofAlphabet:
    alphabetDictionary[alphabet]=alphabetIntegerValue
    alphabetIntegerValue=alphabetIntegerValue+1


#positiveWord=pd.read_csv("/Users/ingridwijaya/Desktop/Analytic Project/positive_words_only.csv")
#negativeWord=pd.read_csv("/Users/ingridwijaya/Desktop/Analytic Project/negative_words_only.csv")

postiveWordArray = []
negativeWordArray = []

positiveWord= client.TwitterIPO.PositiveWords.find({},{"Word":1}).sort([("Word",pymongo.ASCENDING)])
negativeWord= client.TwitterIPO.NegativeWords.find({},{"Word":1}).sort([("Word",pymongo.ASCENDING)])
positiveWordDF =  pd.DataFrame(list(positiveWord))
negativeWordDF =  pd.DataFrame(list(negativeWord))

for alphabet in listofAlphabet:
    
    index=alphabetDictionary[alphabet]-1
    
    alphabetPositiveWords=positiveWordDF[positiveWordDF[positiveWordDF.columns[0]].str.startswith(alphabet.lower())]
    
    alphabetNegativeWords=negativeWordDF[negativeWordDF[negativeWordDF.columns[0]].str.startswith(alphabet.lower())]
    
    #print(alphabetPositiveWords.count())
    #print(alphabetPositiveWords)
    
    postiveWordArray.append(alphabetPositiveWords[alphabetPositiveWords.columns[0]].str.lower().values)
    #print(postiveWordArray)
    
    negativeWordArray.append(alphabetNegativeWords[alphabetNegativeWords.columns[0]].str.lower().values)
    #print(negativeWordArray)

    
cursor = client.TwitterIPO.ProcessedTweets.find()

tweetDataFrame =  pd.DataFrame(list(cursor))

print("Start")

tweetmessages = pd.unique(tweetDataFrame[['UID', 'text']].values)

tknz = TweetTokenizer()

for tweet in tweetmessages:
    
    cleantweet= clean_tweet(tweet[1])
    tokenizetweet= tknz.tokenize(cleantweet)
    
    numberOfPositiveWords=0
    
    numberOfNegativeWords=0
    
    for wordInTweet in tokenizetweet:
        alphabetInWord=wordInTweet[0]
        
        if alphabetInWord in alphabetDictionary:
            alphabetIndex=alphabetDictionary[alphabetInWord]-1
            posArray=postiveWordArray[alphabetIndex]
            negArray=negativeWordArray[alphabetIndex]
            for posword in posArray:
                if(posword==wordInTweet):
                    #print("positive word : ",wordInTweet)
                    numberOfPositiveWords=numberOfPositiveWords+1
                    break
            for negword in negArray:
                if(negword==wordInTweet):
                    #print("negative word : ",wordInTweet)
                    numberOfNegativeWords=numberOfNegativeWords+1
                    break
            #indexOfPositive=binarySearchOfCharacterInWord(alphabetDictionary,posDict,wordInTweet)
    
    sentimentpostivenegative = {"UID": tweet[0],
                                "number_positive_words": numberOfPositiveWords,
                                "number_negative_words": numberOfNegativeWords, 
                                "total_number_of_words":len(tokenizetweet),
                               "ratio" : (numberOfPositiveWords - numberOfNegativeWords) / len(tokenizetweet) }
    
    mongotweetid = client.TwitterIPO["TweetWordCount"].insert_one(sentimentpostivenegative).inserted_id